In [ ]:
import os
import re
import requests

def get_unique_filename(directory, filename, extension):
    original_path = os.path.join(directory, f"{filename}{extension}")
    if not os.path.exists(original_path):
        return original_path
    
    counter = 1
    while os.path.exists(os.path.join(directory, f"{filename}_{counter}{extension}")):
        counter += 1
    
    return os.path.join(directory, f"{filename}_{counter}{extension}")


def extract_urls(directory):
    urls = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            with open(os.path.join(directory, filename), 'r') as file:
                for line in file:
                    match = re.search(r'\bhttps://github.com/DidierRLopes(?:/(?!blob|master|main|tree|commit)[^"/\s]*){2,}\b', line)
                    if match:
                        urls.append(match.group(0))
                        filenames.append(filename)
                    
                    match = re.search(r'\bhttps://github.com/Meg1211(?:/(?!blob|master|main|tree|commit)[^"/\s]*){2,}\b', line)
                    if match:
                        urls.append(match.group(0))
                        filenames.append(filename)
                        
                    match = re.search(r'\bhttps://github-production-user[^"\s]*\b', line)
                    if match:
                        urls.append(match.group(0))
                        filenames.append(filename)
    return urls, filenames

def replace_urls_in_file(filepath, url_map):
    with open(filepath, 'r') as file:
        content = file.read()

    for url, local_path in url_map.items():
        content = content.replace(url, local_path)

    with open(filepath, 'w') as file:
        file.write(content)

urls, filenames = extract_urls('blog')

processed_url = []
last_filename = ""
url_to_local_path_map = {}

for url, filename in zip(urls, filenames):

    if filename != last_filename:
        print("--- " + filename)

    if url not in processed_url:
        response = requests.get(url)
        unique_filename = get_unique_filename("blog/assets/", filename.replace('.md', ''), ".png")
        processed_url.append(url)
        last_filename = filename
        if response.status_code == 200:
            with open(unique_filename, "wb") as file:
                file.write(response.content)
            print(unique_filename)
            url_to_local_path_map[url] = unique_filename
        else:
            print(f"\nFAILED to download {url}\n")

# Replace URLs in the markdown files
for filename in set(filenames):
    replace_urls_in_file(os.path.join('blog', filename), url_to_local_path_map)